In [1]:
import pandas as pd
import matplotlib.pyplot as plt


from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_predict
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import datetime
import calendar

from scipy.stats import pearsonr


In [2]:
# Load data from raw_data
data = np.load('../raw_data/X_train_copernicus.npz', allow_pickle=True)

In [3]:
# Extract features from data
GHI = data['GHI']
CLS = data['CLS']
SAA = data['SAA']
SZA = data['SZA']
DATE = data['datetime']

In [4]:
def n_obs_in_day(observation = 96):
    DATE[observation]
    number_obs_day = 0 
    for i in range(8):
        if DATE[observation+i].day == DATE[observation].day:
            number_obs_day += 1
    return number_obs_day
    
n_obs_in_day()
   

4

In [5]:
# defining time variable 
def arr_time(observation= 96):
    arr_time_day = []
    
    time = datetime.datetime(year=DATE[observation].year,\
                                    month=DATE[observation].month,
                                    day=DATE[observation].day,
                                    hour=DATE[observation].hour,
                                    minute=DATE[observation].minute)
    delta = datetime.timedelta(minutes=15)
    time = time - datetime.timedelta(minutes=60)
    # we're setting the num of obs in day\n",
    observation_max = n_obs_in_day(observation)
    for j in range(observation_max):
        for i in range(8):
            time = time+delta
            arr_time_day.append(time)
    
    arr_time_day  = np.array(arr_time_day)
    return arr_time_day

In [6]:
arr_time().shape

(32,)

In [7]:
def arr_feature(
    feature = CLS,
    observation = 96): 
    arr_day = []
    
    # we're setting the num of obs in day\n",
    observation_max = n_obs_in_day(observation)
    
    for j in range(observation_max):
        for i in range(8):
            arr_day.append(round((feature[observation+j,i,:,:]).mean(),2))
    arr_day = np.array(arr_day)
    return arr_day

    

In [8]:
# defining arrays 
arr_CLS = arr_feature(feature = CLS, observation = 96)
arr_SAA = arr_feature(feature = SAA, observation = 96)
arr_SZA = arr_feature(feature = SZA, observation = 96)
arr_time_obs = arr_time(observation= 96)

In [9]:
# apply the pearsonr()
# CLS and time features
# this analysis is missing 
# I can take a look at this
# https://stackoverflow.com/questions/61517303/how-to-convert-datetime-to-numeric-format-in-python
# so its possible to transform datetime into numeric
corr, _ = pearsonr(arr_CLS, arr_time_obs)
print('Pearsons correlation: %.3f' % corr)


TypeError: unsupported operand type(s) for +: 'float' and 'datetime.datetime'

In [10]:
# apply the pearsonr()
# CLS and SAA features
corr, _ = pearsonr(arr_CLS, arr_SAA)
print('Pearsons correlation: %.3f' % corr)
print('r takes value -1 (negative correlation)')

Pearsons correlation: -1.000
r takes value -1 (negative correlation)


In [11]:
# apply the pearsonr()
# CLS and SZA features
corr, _ = pearsonr(arr_CLS, arr_SZA)
print('Pearsons correlation: %.3f' % corr)
print('Not that much correlation')

Pearsons correlation: 0.282
Not that much correlation


In [13]:
# set as x variable - predictor
x = arr_SAA.reshape((-1, 1))
# set as y variable - response 
y = arr_CLS

In [14]:
# creating an instance 
model = LinearRegression()
# fitting the model 
model.fit(x, y)
# could have used only: model = LinearRegression().fit(x, y)

LinearRegression()

In [16]:
r_sq = model.score(x, y)
print(f"coefficient of determination: {r_sq}")
print('it seems that SAA will determine the GHI under clean sky')

coefficient of determination: 0.9997187604559593
it seems that SAA will determine the GHI under clean sky


In [17]:
# now lets do the same for SZA
# set as x variable - predictor
x2 = arr_SZA.reshape((-1, 1))
# set as y variable - response 
y2 = arr_CLS

In [18]:
# instance and fitting
model2 = LinearRegression().fit(x2, y2)

In [19]:
r_sq2 = model2.score(x2, y2)
print(f"coefficient of determination: {r_sq2}")
print('it seems that SZA will NOT determine the GHI under clean sky')
# we can look at the solar elevation


coefficient of determination: 0.07968160356161635
it seems that SZA will NOT determine the GHI under clean sky


In [20]:
# defining solar elevation as per Federico definition 
def arr_sol_elevation(
    feature = SZA, # SHOULD ALWAYS BE SOLAR ZENITH ANGLE
    observation = 96): 
    arr_sol_elev_day = []
    
    # we're setting the num of obs in day\n",
    observation_max = n_obs_in_day(observation)
    
    for j in range(observation_max):
        for i in range(8):
            arr_sol_elev_day.append(90-(round((feature[observation+j,i,:,:]).mean(),2)))   
    arr_sol_elev_day = np.array(arr_sol_elev_day)
    return arr_sol_elev_day 

In [21]:
# setting Solar Angle Elevation as SAE
arr_SAE = arr_sol_elevation(feature = SZA, observation = 96)

In [27]:
# Now lets set the 3rd model 
# set as x3 variable - predictor
x3 = arr_SAE.reshape((-1, 1))
# set as y3 variable - response 
y3 = arr_CLS

In [28]:
# instance and fitting
model3 = LinearRegression().fit(x3, y3)


In [29]:
r_sq3 = model3.score(x3, y3)
print(f"coefficient of determination: {r_sq3}")
print('it seems that SAE will determine the GHI under clean sky')


coefficient of determination: 0.9913386821435878
it seems that SAE will determine the GHI under clean sky


### Providing the same analysis on 24 of March

In [30]:
# defining variables
n_obs_in_day(observation = 536)
# defining arrays 
arr_CLS = arr_feature(feature = CLS, observation = 536)
arr_SAA = arr_feature(feature = SAA, observation = 536)
arr_SZA = arr_feature(feature = SZA, observation = 536)
arr_SAE = arr_sol_elevation(feature = SZA, observation = 536)


In [32]:
# set as x variable - predictor
x4 = arr_SAA.reshape((-1, 1))
x5 = arr_SZA.reshape((-1, 1))
x6 = arr_SAE.reshape((-1, 1))
# set as y variable - response 
y4 = arr_CLS

# instance and fitting
model4 = LinearRegression().fit(x4, y4)
model5 = LinearRegression().fit(x5, y4)
model6 = LinearRegression().fit(x6, y4)


In [33]:
r_sq4 = model4.score(x4, y4)
print(f"coefficient of determination: {r_sq4}")
print('it seems that SAA will determine the GHI under clean sky')

coefficient of determination: 0.9913386821435878
it seems that SAA will determine the GHI under clean sky


In [34]:
r_sq5 = model5.score(x5, y4)
print(f"coefficient of determination: {r_sq5}")
print('it seems that SZA will NOT determine the GHI under clean sky')

coefficient of determination: 0.0027612355508339625
it seems that SZA will NOT determine the GHI under clean sky


In [35]:
r_sq6 = model6.score(x6, y4)
print(f"coefficient of determination: {r_sq6}")
print('it seems that SAE will NOT determine the GHI under clean sky')

coefficient of determination: 0.0027612355508338515
it seems that SAE will NOT determine the GHI under clean sky


## Conclusion

### Solar Azimuth Angle seems to be highly correlated with GHI under clean sky 
### Solar Zenith Angles DOES NOT seem